In [58]:
import pandas as pd
import os
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import locale
import calendar
import openpyxl
import shutil
import xlwings as xw
import traceback
import numpy as np
pd.options.mode.chained_assignment = None  # desativa o aviso de SettingWithCopy

###-----------------------------------------------Variáveis com datas----------------------------------------------###
locale.setlocale(locale.LC_TIME, 'pt_BR.utf-8')
hoje = datetime.now()

# Se hoje é segunda-feira, recuar 2 dias (para sábado); caso contrário, recuar 1 dia (para ontem)
if hoje.strftime('%A') == 'segunda-feira':
    date_time = hoje - timedelta(days=2)
else:
    date_time = hoje - timedelta(days=1)


data_hoje = hoje.date()

data_hoje_aaaamm = str(date_time.year) + '0' + str(date_time.month)
data_hoje_mm_aaaa = '0' + str(date_time.month) + '-' + str(date_time.year) 

data_ontem = date_time
data_ontem_aaaamm =  str(data_ontem.year) + '0' + str(data_ontem.month)

data_mes_anterior_mm_aaaa = '0' + str((data_ontem - relativedelta(months=1)).month) +'-' + str((data_ontem - relativedelta(months=1)).year)
data_mes_anterior_aaaamm = str((data_ontem - relativedelta(months=1)).year) + '0' + str((data_ontem - relativedelta(months=1)).month)
data_dois_meses_anteriores_aaaamm = str((data_ontem - relativedelta(months=2)).year) + '0' + str((data_ontem - relativedelta(months=2)).month)
data_tres_meses_anteriores_aaaamm = str((data_ontem - relativedelta(months=3)).year) + '0' + str((data_ontem - relativedelta(months=3)).month)

dia_anterior_dd_mm_aa = str(data_ontem.day) + '-' + str(data_ontem.month) + '-' + str(data_ontem.year)
dia_anterior_mm_aa = str(data_ontem.month) + '-' + str(data_ontem.year)

abrev_mes_ano_atual = (date_time - relativedelta(days=1)).strftime('%b/%y')
abrev_mes_ano_passado = (date_time - relativedelta(years=1)).strftime('%b/%y')
mes_anterior_abrev_mes_ano = (date_time - relativedelta(months=1)).strftime('%b/%y')
dois_meses_anteriores_abrev_mes_ano = (date_time - relativedelta(months=2)).strftime('%b/%y')
tres_meses_anteriores_abrev_mes_ano = (date_time - relativedelta(months=3)).strftime('%b/%y')

QNT_dias_no_mes = calendar.monthrange(date_time.year, date_time.month)[1]


In [59]:
'-----------------//------------------Funções--------------------------------//-------------------'

def remove_colunas(df, colunas: list):
    df.drop(columns=colunas, inplace=True)

def remove_valor(df: pd.DataFrame, colunas: list, valores_excluidos: list) -> pd.DataFrame:
    """
    Substitui por None os valores indesejados nas colunas especificadas.

    Parâmetros:
    - df: DataFrame original.
    - colunas: Lista de colunas onde os valores devem ser substituídos.
    - valores_excluidos: Lista de valores a serem substituídos por None.

    Retorna:
    - DataFrame com os valores substituídos.
    """
    for coluna in colunas:
        df[coluna] = df[coluna].apply(lambda x: None if x in valores_excluidos else x)
    return df

def limpaterminal():    
    os.system('cls')

def calcula_pocentagem(dividendo, divisor):
    try:
        num1 = float(str(dividendo).replace(',', '.'))
        num2 = float(str(divisor).replace(',', '.'))
        if num2 == 0:
            return '0%'
        valor_porcentagem =  num1 / num2
        return valor_porcentagem
    except:
        return 'N/A' 
    
def adiciona_info_painel(dataframe):
    novas_linhas = []
    tamanho = len(lista_filiais_unicas)
    contador_filiais_analisadas = 0

#Atualizar elementos que não precisam do indicador para funcionar, assim evitando uso desnecessário da funcao pega_info
    for filial in lista_filiais_unicas:
        def pega_info(indicador, coluna):
            filtro = dataframe[(dataframe['INDICADOR'] == indicador) & (dataframe['FILIAL'] == filial)]
            return filtro[coluna].iloc[0] if not filtro.empty else None

        nova_linha = {
            "FILIAL": filial,
            "PRODUÇÃO TOTAL": pega_info('PRODUÇÃO TOTAL', 'REAL_MES'),
            "FATURAMENTO TOTAL": pega_info('FATURAMENTO TOTAL', 'REAL_MES'),
            "D-1": pega_info('PRODUÇÃO TOTAL', 'REAL_DIA'),
            #NAS SEGUNDAS DEVEMOS GERAR O ARQUIVO PELO PAINEL E REFERENCIAR ATRAVÉS DE PROCV
            #FUTURAMENTE ATUALIZAR PUXANDO INFOS DA PLANILHA PRODUÇÃO
            "MARGEM": pega_info('MARGEM BRUTA TOTAL', 'REAL_MES'),
            "GESTOR DE CONTATOS": pega_info('GESTOR DE CONTATOS', 'REAL_MES'),
            "VENDEX": pega_info('VISITAS VENDA EXTERNA - QTDE', 'REAL_MES'),
            "CARTÕES ATIVADOS": pega_info('CARTÕES ATIVADOS - QTDE', 'REAL_MES'),
            "CARTÕES ATIVADOS META": pega_info('CARTÕES ATIVADOS - QTDE', 'META_MES'),
            "METAORC": pega_info('PRODUÇÃO TOTAL', 'ORC_MES'),
            "META": pega_info('PRODUÇÃO TOTAL', 'META_MES'),
            "FIGITAL": pega_info('FIGITAL', 'REAL_MES'),
            "FIGITAL ORC": pega_info('FIGITAL', 'ORC_MES'),
            "EP": pega_info('ENCARGOS', 'REAL_MES'),
            "V+": pega_info('V+ R$', 'REAL_MES'),
            "CARTÕES ATIVADOS QTDE": pega_info('CARTÕES ATIVADOS - QTDE', 'REAL_MES'),
            "CARTÕES ATIVADOS - QTDE ORC": pega_info('CARTÕES ATIVADOS - QTDE', 'ORC_MES'),
            'SEGUROS': pega_info('SEGUROS TOTAL - R$', 'REAL_MES'),
            'PRODUÇÃO TOTAL ORC MES': pega_info('PRODUÇÃO TOTAL', 'ORC_MES'), #OTAVIO
            'EMISSORA': pega_info('PRODUÇÃO TOTAL', 'EMISSORA'),
            
            "CONSTRUCAO%": calcula_pocentagem(pega_info('CONSTRUCAO', 'REAL_MES'), pega_info('FATURAMENTO TOTAL', 'REAL_MES')),
            "CONSTRUCAO": pega_info('CONSTRUCAO', 'REAL_MES'),
            "M.BRUTO%": calcula_pocentagem(pega_info('CONST. MAT. BRUTO', 'REAL_MES'), pega_info('FATURAMENTO TOTAL', 'REAL_MES')),
            "M.BRUTO": pega_info('CONST. MAT. BRUTO', 'REAL_MES'),
            "ELETRO%": calcula_pocentagem(pega_info('ELETRO', 'REAL_MES'), pega_info('PRODUÇÃO TOTAL', 'REAL_MES')), #OTAVIO - É faturamento mesmo? Antigo relatorio usa produção total
            "ELETRO": pega_info('ELETRO', 'REAL_MES'),
            "MOVEIS%": calcula_pocentagem(pega_info('MOVEIS', 'REAL_MES'), pega_info('PRODUÇÃO TOTAL', 'REAL_MES')), #OTAVIO - Porque ORC e não REAL? Antigo relatorio usa produção total real mes
            "MOVEIS": pega_info('MOVEIS', 'REAL_MES'),
            "FAT ATING MÊS": calcula_pocentagem(pega_info('FATURAMENTO TOTAL', 'REAL_MES'), pega_info('FATURAMENTO TOTAL', 'ORC_MES')),
            f"FAT ATING {mes_anterior_abrev_mes_ano}": calcula_pocentagem(pega_info('FATURAMENTO TOTAL', f'REAL_MES_{mes_anterior_abrev_mes_ano}'), pega_info('FATURAMENTO TOTAL', f'ORC_MES_{mes_anterior_abrev_mes_ano}')),
            f"FAT ATING {dois_meses_anteriores_abrev_mes_ano}": calcula_pocentagem(pega_info('FATURAMENTO TOTAL', f'REAL_MES_{dois_meses_anteriores_abrev_mes_ano}'), pega_info('FATURAMENTO TOTAL', f'ORC_MES_{dois_meses_anteriores_abrev_mes_ano}')),
            f"FAT ATING {tres_meses_anteriores_abrev_mes_ano}": calcula_pocentagem(pega_info('FATURAMENTO TOTAL', f'REAL_MES_{tres_meses_anteriores_abrev_mes_ano}'), pega_info('FATURAMENTO TOTAL', f'ORC_MES_{tres_meses_anteriores_abrev_mes_ano}')),
            "PROD ATING MÊS": calcula_pocentagem(pega_info('PRODUÇÃO TOTAL', 'REAL_MES'), pega_info('PRODUÇÃO TOTAL', 'ORC_MES')),
            "CARTÕES ATIVADOS%": calcula_pocentagem(pega_info('CARTÕES ATIVADOS - QTDE', 'REAL_MES'), pega_info('CARTÕES ATIVADOS - QTDE', 'ORC_MES')),
            "FIGITAL%": calcula_pocentagem(pega_info('FIGITAL', 'REAL_MES'), pega_info('FIGITAL', 'ORC_MES')),
            "ENCARGOS%":calcula_pocentagem(pega_info('ENCARGOS', 'REAL_MES'), pega_info('ENCARGOS', 'ORC_MES')),
            "CARTÕES NOVOS%": calcula_pocentagem(pega_info('CARTÕES ATIVADOS - QTDE', 'REAL_MES'), pega_info('CARTÕES ATIVADOS - QTDE', 'ORC_MES')),
            "SEGUROS%": calcula_pocentagem(pega_info('SEGUROS TOTAL - R$', 'REAL_MES'), pega_info('SEGUROS TOTAL - R$', 'ORC_MES')), #CRIADO - OTAVIO
            "EP%": calcula_pocentagem(pega_info('ENCARGOS CRÉDITO PESSOAL LÍQUIDO DE DESCONTOS', 'REAL_MES'), pega_info('ENCARGOS CRÉDITO PESSOAL LÍQUIDO DE DESCONTOS', 'ORC_MES')), #CRIADO - OTAVIO
            "CAB%": calcula_pocentagem(pega_info('PRODUÇÃO TOTAL', 'REAL_MES'), pega_info('PRODUÇÃO TOTAL', 'ORC_MES')),


            "GRUPO": pega_info('PRODUÇÃO TOTAL', 'GRUPO'),
            
            "NOME FILIAL": pega_info('PRODUÇÃO TOTAL', 'NOME_FILIAL'),
            "UF": pega_info('PRODUÇÃO TOTAL', 'UF'),
            "REGIÃO": pega_info('PRODUÇÃO TOTAL', 'REGIÃO'),
            "DT_ABERT": pega_info('PRODUÇÃO TOTAL', 'DT_ABERT'),
            "COD_REGIÃO": pega_info('PRODUÇÃO TOTAL', 'COD_REGIÃO'),
        }

        novas_linhas.append(nova_linha)
        contador_filiais_analisadas += 1
        print(f'Adicionando informações do painel mensal {contador_filiais_analisadas}/{tamanho}   ', end='\r')

    
    return pd.DataFrame(novas_linhas)   


def preencher_planilhas(arquivo_copia, dados_por_aba):
    app = xw.App(visible=False)
    arquivo = app.books.open(arquivo_copia)

    try:
        for nome_aba, insercoes in dados_por_aba.items():
            aba = arquivo.sheets[nome_aba]
            for linha_inicio, coluna_inicio, df in insercoes:
                destino = aba.range((linha_inicio, coluna_inicio))

                if destino.merge_cells:
                    destino = destino.merge_area[0, 0]

                destino.value = df.values

        arquivo.save()
    finally:
        arquivo.close()
        app.quit()

'-----------------//------------------Funções--------------------------------//-------------------'

'-----------------//------------------Funções--------------------------------//-------------------'

In [60]:
pedacos_filtrados = []

for chunk in pd.read_csv(
     r"O:\Geral\producao.csv",
    sep=';',
    encoding='latin1',
    usecols=['FILIAL', 'DATA', 'VALOR'],  # ainda carrega a coluna DATA
    dtype={'FILIAL': 'string', 'DATA': 'string', 'VALOR': 'string'},
    chunksize=4000
):
    filtrado = chunk[chunk['DATA'] == data_ontem.strftime('%d/%m/%Y')]
    
    filtrado['VALOR'] = (
        filtrado['VALOR']
        .str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype('float32')
    )

    filtrado = filtrado.drop(columns='DATA')

    pedacos_filtrados.append(filtrado)

df_filtrado = pd.concat(pedacos_filtrados, ignore_index=True)

In [61]:
df_agrupado = df_filtrado.groupby('FILIAL', as_index=False)['VALOR'].sum()


In [66]:
hoje.strftime('%A')

'segunda-feira'

In [65]:
df_agrupado[df_agrupado['FILIAL'] == '3']

,FILIAL,VALOR
184,3,9056.030273
